In [4]:
import requests
import ssl
import string
from urllib3 import poolmanager
from bs4 import BeautifulSoup

class TLSAdapter(requests.adapters.HTTPAdapter):

    def init_poolmanager(self, connections, maxsize, block=False):
        """Create and initialize the urllib3 PoolManager."""
        ctx = ssl.create_default_context()
        ctx.set_ciphers('DEFAULT@SECLEVEL=1')
        self.poolmanager = poolmanager.PoolManager(
                num_pools=connections,
                maxsize=maxsize,
                block=block,
                ssl_version=ssl.PROTOCOL_TLS,
                ssl_context=ctx)

session = requests.session()
session.mount('https://', TLSAdapter())

player_links = {}

for alphabet in string.ascii_lowercase:
    response = session.get(f'https://npb.jp/bis/eng/players/active/index_{alphabet}.html')
    soup = BeautifulSoup(response.content, 'html.parser')
    for elem in soup.select('#pl_result_list a'):
        link = elem.attrs['href']
        if link == '#':
            continue
            
        surname, name = elem.select('.name')[0].text.split(',', 1)
        name = f'{name.strip()}-{surname.strip()}'.replace(' ', '-')
        player_links[name] = link

In [5]:
avatars = {}

for name, player_link in player_links.items():
    response = session.get(f'https://npb.jp{player_link}')
    soup = BeautifulSoup(response.content, 'html.parser')
    elem = soup.select('div#pc_v_photo img')[0]
    avatars[name] = elem.attrs['src']

In [6]:
import os
output_dir = f'../outputs/npb'
os.makedirs(output_dir, exist_ok=True)

for name, link in avatars.items():
    response = session.get(link)
    with open(os.path.join(output_dir, f'{name}.jpg'), 'wb') as fp:
        fp.write(response.content)
    